In [168]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from cryptography.fernet import Fernet
import re
import datetime as dt

In [169]:
dfPrinc = pd.read_csv('../../dados/dfPrincipal.csv')

In [170]:
dfPrinc.head()

,Unnamed: 0,SEXO,DT_NASCIMENTO,FORMA_INGRESSO,FORMA_EVASAO,PERIODO_INGRESSO,DT_EVASAO,PERIODO_EVASAO,PER_EVASAO_FORMAT,ANO_EVASAO,PER_INGRESSO_FORMAT,ANO_INGRESSO
0,0,M,1982-02-09,VE - Vestibular,CON - Curso concluído,2,2009-09-14,2.0,2009.2,2009.0,2002.2,2002
1,1,M,1977-05-09,VE - Vestibular,ABA - Abandono do curso,1,2005-03-12,2.0,2005.2,2005.0,2004.1,2004
2,2,M,1985-03-05,VE - Vestibular,Não identificada (vide pasta do aluno),1,2003-01-06,1.0,2003.1,2003.0,2003.1,2003
3,3,M,1988-07-18,SISU Ampla Concorrencia,CON - Curso concluído,2,2018-03-23,2.0,2017.2,2017.0,2013.2,2013
4,4,M,2002-07-02,"SISU Escola Pública, Indep. de Renda : Preto e...",Sem evasão,1,NaN,NaN,nan.,NaN,2020.1,2020


### Analise Dados Alunos



--- 

### Grupos de Alunos

| /                           |Todos os Alunos | Antes de 2013 | De 2013 até 2020 | De 2020 até 2021 
| --------------------------  | -------------- |  -----------  | -----------      | -----------     | 
Análise da % de conclusão     | x              |  -----------  | -----------      | -----------     | 
Análise da % de genero        | x              |  -----------  | -----------      | -----------     | 
Análise do perfil de idade    | x              |  -----------  | -----------      | -----------     | 
Análise da distancia          | -------------- |  -----------  | -----------      | -----------     | 
Análise do rendimento escolar | -------------- |  -----------  | -----------      | -----------     | 
Análise da % de conclusão     | -------------- |  -----------  | -----------      | -----------     | 







## Análise da % de conclusão(Todos os Alunos)

In [4]:
evasoesDF = dfPrinc.groupby(['ANO_EVASAO','FORMA_EVASAO'], dropna=False).size().reset_index(name='ALUNOS')

In [5]:
evasoesDF['FORMA_EVASAO'] = evasoesDF.FORMA_EVASAO.str.replace('ABA - Abandono do curso', 'Evasão') 
evasoesDF['FORMA_EVASAO'] = evasoesDF.FORMA_EVASAO.str.replace('APO - Aluno Especial - Disciplina Isolada', 'Evasão') 
evasoesDF['FORMA_EVASAO'] = evasoesDF.FORMA_EVASAO.str.replace('APO -Conclusão a Mobilidade Acadêmica Externa - ANDIFES', 'Evasão') 
evasoesDF['FORMA_EVASAO'] = evasoesDF.FORMA_EVASAO.str.replace('APO -Conclusão da Mobilidade Acadêmica Externa - IAE/IESCE', 'Evasão') 
evasoesDF['FORMA_EVASAO'] = evasoesDF.FORMA_EVASAO.str.replace('Desistência SiSU', 'Evasão') 
evasoesDF['FORMA_EVASAO'] = evasoesDF.FORMA_EVASAO.str.replace('JUB - Jubilamento', 'Evasão') 
evasoesDF['FORMA_EVASAO'] = evasoesDF.FORMA_EVASAO.str.replace('CAN - Cancelamento Geral do curso', 'Evasão') 
evasoesDF['FORMA_EVASAO'] = evasoesDF.FORMA_EVASAO.str.replace('CON - Curso concluído', 'Concluído') 
evasoesDF['FORMA_EVASAO'] = evasoesDF.FORMA_EVASAO.str.replace('Sem evasão', 'Cursando') 

In [6]:
evasoesDF.loc[evasoesDF['FORMA_EVASAO'] == 'Não identificada (vide pasta do aluno)' , 'FORMA_EVASAO'] = 'Evasão'

In [7]:
evasoesDF = evasoesDF.groupby(['FORMA_EVASAO'] , as_index=False)['ALUNOS'].sum() 

In [8]:
# Mostra porcentagem de alunos
evasoesDF['PERCENT'] =  (100. * evasoesDF['ALUNOS'] / evasoesDF['ALUNOS'].sum()).round(0)


In [9]:
evasoesDF

,FORMA_EVASAO,ALUNOS,PERCENT
0,Concluído,335,27.0
1,Cursando,418,33.0
2,Evasão,507,40.0


In [10]:
evasoesDF.to_csv ('../../dados/evasoesDF.csv', index = False, header=True)
print(evasoesDF)

  FORMA_EVASAO  ALUNOS  PERCENT
0    Concluído     335     27.0
1     Cursando     418     33.0
2       Evasão     507     40.0


---

## Análise da % de gênero

In [9]:
generoDF = dfPrinc.groupby(['SEXO'], dropna=False).size().reset_index(name='ALUNOS')

In [10]:
generoDF['PERCENT'] =  (100. * generoDF['ALUNOS'] / generoDF['ALUNOS'].sum()).round(0)

In [13]:
generoDF.loc[(generoDF['SEXO'] == 'M'),'SEXO']='Masculino'
generoDF.loc[(generoDF['SEXO'] == 'F'),'SEXO']='Feminino'

In [14]:
generoDF

,SEXO,ALUNOS,PERCENT
0,Feminino,238,19.0
1,Masculino,1022,81.0


In [15]:
generoDF.to_csv ('../../dados/generoDF.csv', index = False, header=True)
print(generoDF)

        SEXO  ALUNOS  PERCENT
0   Feminino     238     19.0
1  Masculino    1022     81.0


## Análise do perfil de idade


In [193]:
dfPrinc['DT_NASCIMENTO'] = pd.to_datetime(dfPrinc['DT_NASCIMENTO'])


In [194]:
#colunas = ['SEXO', 'FORMA_INGRESSO','FORMA_EVASAO','PERIODO_INGRESSO','DT_EVASAO','PERIODO_EVASAO','PER_EVASAO_FORMAT','ANO_EVASAO',
#           'PER_INGRESSO_FORMAT']
colunas = ['DT_NASCIMENTO','ANO_INGRESSO']
idadeDF = dfPrinc.filter(colunas)


In [195]:
arrayDatas = dfPrinc['DT_NASCIMENTO'].array
arrayIngresso = dfPrinc['ANO_INGRESSO'].array
arrayIdadeFormatado = []
def getIdadeIngresso(DT_NASCIMENTO, ANO_INGRESSO):
    return arrayIdadeFormatado.append(ANO_INGRESSO - DT_NASCIMENTO.year)
    

In [196]:
for dt,ing in zip(arrayDatas,arrayIngresso):
    getIdadeIngresso(dt, ing)

In [197]:
idadeDF['IDADE'] = arrayIdadeFormatado
idadeDF = idadeDF.fillna(0)
idadeDF['IDADE'] = idadeDF['IDADE'].astype(int)

In [198]:
idadeDF

,DT_NASCIMENTO,ANO_INGRESSO,IDADE
0,1982-02-09 00:00:00,2002,20
1,1977-05-09 00:00:00,2004,27
2,1985-03-05 00:00:00,2003,18
3,1988-07-18 00:00:00,2013,25
4,2002-07-02 00:00:00,2020,18
...,...,...,...
1255,2000-07-22 00:00:00,2020,20
1256,1996-02-07 00:00:00,2014,18
1257,1992-03-29 00:00:00,2009,17
1258,1991-02-17 00:00:00,2009,18


#### Fazendo Group By

In [199]:
idadeDF = idadeDF.groupby(['IDADE'], dropna=True).size().reset_index(name='ALUNOS')

In [200]:
idadeDF

,IDADE,ALUNOS
0,0,1
1,17,20
2,18,253
3,19,300
4,20,167
5,21,106
6,22,58
7,23,57
8,24,64
9,25,43


In [201]:
idadeDF = idadeDF.drop(0)

In [202]:
idadeDF['PERCENT'] =  (100. * idadeDF['ALUNOS'] / idadeDF['ALUNOS'].sum()).round(1)

In [203]:
idadeDF.set_index('IDADE', inplace=True)

In [204]:
nidadeDF = idadeDF.T

In [205]:
nidadeDF

IDADE,17,18,19,20,21,22,23,24,25,26,...,50,51,52,53,54,55,56,57,58,64
ALUNOS,20.0,253.0,300.0,167.0,106.0,58.0,57.0,64.0,43.0,29.0,...,1.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0
PERCENT,1.6,20.1,23.8,13.3,8.4,4.6,4.5,5.1,3.4,2.3,...,0.1,0.1,0.1,0.2,0.2,0.1,0.1,0.1,0.1,0.1


In [206]:
idadeDF
nidadeDF['17-21'] = nidadeDF.iloc[:,0:4].sum(axis=1)
#df.loc[len(df.index)] = ['Amy', 89, 93]

#nidadeDF = idadeDF.T 
nidadeDF['22-26'] = nidadeDF.iloc[:,5:9].sum(axis=1)
nidadeDF['27-31'] = nidadeDF.iloc[:,10:14].sum(axis=1)
nidadeDF['32-36'] = nidadeDF.iloc[:,15:19].sum(axis=1)
nidadeDF['37-41'] = nidadeDF.iloc[:,20:24].sum(axis=1)
nidadeDF['< 41 > 51'] = nidadeDF.iloc[:,25:34].sum(axis=1)
nidadeDF['< 51'] = nidadeDF.iloc[:,35:42].sum(axis=1)


In [207]:
nidadeDF = nidadeDF.drop(nidadeDF.iloc[:, 0:42],axis = 1)

In [208]:
nidadeDF = nidadeDF.T

In [210]:
nidadeDF

,ALUNOS,PERCENT
IDADE,,
17-21,740.0,58.8
22-26,222.0,17.6
27-31,76.0,6.1
32-36,35.0,2.8
37-41,11.0,0.9
< 41 > 51,15.0,1.4
< 51,9.0,0.9


In [211]:
nidadeDF.to_csv ('../../dados/idadeDF.csv', index = True, header=True)
print(idadeDF)

       ALUNOS  PERCENT
IDADE                 
17         20      1.6
18        253     20.1
19        300     23.8
20        167     13.3
21        106      8.4
22         58      4.6
23         57      4.5
24         64      5.1
25         43      3.4
26         29      2.3
27         19      1.5
28         26      2.1
29         19      1.5
30         12      1.0
31          8      0.6
32         16      1.3
33          7      0.6
34          8      0.6
35          4      0.3
36          6      0.5
37          4      0.3
38          3      0.2
39          2      0.2
40          2      0.2
41          1      0.1
42          2      0.2
43          1      0.1
44          1      0.1
46          2      0.2
47          3      0.2
48          2      0.2
49          2      0.2
50          1      0.1
51          1      0.1
52          1      0.1
53          2      0.2
54          2      0.2
55          1      0.1
56          1      0.1
57          1      0.1
58          1      0.1
64         